# Web Scraping Oscar Winning movies from IMDB (www.imdb.com)

![oscar_banner](https://i.imgur.com/nz7I0zr.png)

A project work as part of the course zero to data analyst from Jovian. \
**IMDB**: International Movie Database - is the world's most popular and authoritative source for movie, TV and celebrity content. Find ratings and reviews for the newest movie and TV shows. \
What is web scraping: 
web scraping is a process extracting information from a website programmitcally using code. In this initiative we will attempt to extract information using python and some of the libraries known for web scraping called **requests** and **beautiful soup**.

Following are the steps on how to extract the information:

1. Load the libraries needed for the project 
2. Download the page using requests
3. Parse the html source code using BeautifulSoup4
4. Extract the information and the data that we are trying to about the movies
5. Extract and combine data from multiple pages from the portal
6. Compile extracted information into python data objects
7. Finally, write and save the info into CSV files

By the end of this project we will have the information about Oscar Winning Movies in a certain period into files This is the info we will gather about the movies: movie title,year,IMDB Rating,movie rating,movie duration, movie Genre, Metascore, Votes, Gross USD.

So, lets get to it!!

**How to run the code**

Content present in this Jupyter notebook is executable by any user and is hosted on Jovian. \
Running using free online resources (1-click, recommended):\
The easiest way to start executing the code is to click the Run button at the top of this page and select Run on Binder. You can also select "Run on Colab" or "Run on Kaggle", but you'll need to create an account on Google Colab or Kaggle to use these platforms.

### Load the libraries needed for the project

In [3]:
import requests #packages that is used to download the content from web
import urllib # packages that is used to work with URL libraries
import requests #package built to make HTTP requests user friendly
import os # package used for file process
import re # package for regular expression - best to have it dont know if it is required or not
from bs4 import BeautifulSoup #a Python library for pulling data out of HTML and XML files
import pandas as pd # the omnipresent of all python to work with dataframes
requests.__version__ # check the version of the package

'2.19.1'

### Download the page using requests

With the goal of getting Oscars dataset with as minimal requests as possible sent to IMDB, so not to bog down their servers with a lot of people like me trying to learn about web scraping - lets start slow. 

Go to the portal and get retrieve using advanced search for all the movies that won Oscar for the year 2017 (test year) so we can walk thru each attribut that we want to collect and then build a final dataset

Go to Advanced search on the imdb portal 

![search_banner](https://i.imgur.com/DcpcASO.png)

This is how the link looks like: https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&groups=oscar_winner \
As you can see after the ? on the link there are attributes that are of interest to us:
release_date and oscar_winner

In [4]:
#link to all the movies that won Oscar for 2017
oscar_url = 'https://www.imdb.com/search/title/?release_date=2017&groups=oscar_winner' 
oscar_resp = requests.get(oscar_url) #get the content of the url from the request package.
print(oscar_resp.text[:200]) # look at the snapshot of the content



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         

        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatibl


### Parse the html source code using BeautifulSoup4

A request is basically a user call function to the website to access the content from the portal.

From the few lines shown from response its easy to decipher that the content accessed is a HTML document. Once we understand the navigation and access of the HTML tags and content on one page it follows the same structure for all the pages and makes it easy to build a script.

Right-click on a web page element for example on the first movie in the list and 'inspect' to see the structure and the way the page is built within HTML. Here we are looking at the title of first movie. 

![parse_html](https://i.imgur.com/1Tprjux.png)


> Time to use BeautifulSoup to parse and work with HTML tags

In [5]:
oscar_soup = BeautifulSoup(oscar_resp.text, 'html.parser') # with the response text created - parse the html

Now, its time to look at the page and see what is the best way to extract the list of movies for 2017 into a container with the script.

![2017_list](https://i.imgur.com/xPdleg2.png)

Looks like there are 14 titles with Oscar win for 2017

As you can see all the titles are in lister-item more-advanced class of div element:

![list of search movies](https://i.imgur.com/5kSZALg.png)

In [6]:
oscar_containers = oscar_soup.find_all("div",{"class" : "lister-item mode-advanced"}) # get all the containers 
print('Containers is of type: ',type(oscar_containers)) # check the type 
print('Number of movies in the container:',len(oscar_containers)) # number of movies in the container

('Containers is of type: ', <class 'bs4.element.ResultSet'>)
('Number of movies in the container:', 14)


### Extract the information and the data that we are trying to about the movies

**Before we build through the whole script - lets work thru the attributes we want and how to get those attributes with one movie from the container**

For the project, goal is to extract the following detail for the movies list:
- name 
- year
- rating
- certificate
- duration 
- genre 
- metascore
- votes
- gross

In [7]:
oscar_1 = oscar_containers[0] # get the first movie from the container
print('What is the data type of movie:',type(oscar_1)) # observe the tag of the first movie
print('details of the movie: ', len(oscar_1))

('What is the data type of movie:', <class 'bs4.element.Tag'>)
('details of the movie: ', 7)


*As you can see that there is lot of info to unpack so lets go step by step and see where the attributes that we need*

In [8]:
oscar_1.div # div tag or a section or division within HTML

<div class="lister-top-right">\n<div class="ribbonize" data-caller="filmosearch" data-tconst="tt1856101"></div>\n</div>

In [9]:
oscar_1.a # a or anchor tag defines the href which is hypertext reference or link for the content on the page

<a href="/title/tt1856101/"> <img alt="Blade Runner 2049" class="loadlate" data-tconst="tt1856101" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BNzA1Njg4NzYxOV5BMl5BanBnXkFtZTgwODk5NjU3MzI@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>\n</a>

**movie title** 

Looks like the title is hidden within div, h3, a 

![movie_title](https://i.imgur.com/axsCnaB.png)

In [10]:
oscar_1.h3 # access h3 content 

<h3 class="lister-item-header">\n<span class="lister-item-index unbold text-primary">1.</span>\n<a href="/title/tt1856101/">Blade Runner 2049</a>\n<span class="lister-item-year text-muted unbold">(2017)</span>\n</h3>

In [11]:
oscar_1.h3.a # access he anchor tag and now its getting close to get the movie title

<a href="/title/tt1856101/">Blade Runner 2049</a>

In [12]:
movie_name = oscar_1.h3.a.text # now access just the text part from the tag
movie_name

u'Blade Runner 2049'

**movie year** \
One step done, now to get the Year which happened to be in the 'span' tag.
Using the *find* method we can get first occurence by matching the class *lister-item-year text-muted unbold* from the picture:

![movie_year](https://i.imgur.com/IHJq5DH.png)

In [13]:
movie_year = oscar_1.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
movie_year

u'(2017)'

**movie rating**

![movie_rating](https://i.imgur.com/bsfEzYP.png)

In [14]:
oscar_1.strong # attribute for rating seems to be pretty easy contained witin *strong* tag

<strong>8.0</strong>

In [15]:
# convert to float as expecting to see some decimal places for rating
movie_rating = float(oscar_1.strong.text) 
movie_rating

8.0

**movie certification**

![movie_details](https://i.imgur.com/XAacbCX.png)

In [16]:
# now to get the rest attributes group in p tag with different class
oscar_1.find('span', class_ = 'certificate')

<span class="certificate">UA</span>

In [17]:
movie_certificate = oscar_1.find('span', class_ = 'certificate').text
movie_certificate

u'UA'

`Not all movies are rated so lets do a helper function to put a 'Not Rated' where not available`

In [18]:
def get_certificate(oscar):
    # cases where there is no certificate assigned - lets give default as Not Rated
    if oscar.find('span', class_ = 'certificate') is not None:
        cert = oscar.find('span', class_ = 'certificate').text
    else:
        cert = 'Not Rated'
    return cert

In [19]:
get_certificate(oscar_1)

u'UA'

**movie runtime**

In [20]:
movie_runtime = oscar_1.find('span', class_ = 'runtime').text
movie_runtime

u'164 min'

**movie genre**

In [21]:
movie_genre = oscar_1.find('span', class_ = 'genre').text
movie_genre

u'\nAction, Drama, Mystery            '

In [22]:
movie_genre = movie_genre.strip()
movie_genre

u'Action, Drama, Mystery'

**movie vote count and gross earned**

To get the vote count and also the gross amount of the movie

![movie_gross_vote](https://i.imgur.com/COYD2of.png)

In [23]:
v_g_dtl = oscar_1.findAll('span', attrs = {'name' : 'nv'})
movie_votes = v_g_dtl[0]['data-value']
print(movie_votes)

492917


In [24]:
movie_gross = v_g_dtl[1].text
print(movie_gross)

$92.05M


In [25]:
#get vote count 
oscar_1.findAll('span', attrs = {'name' : 'nv'})[0]['data-value']

u'492917'

In [26]:
# get gross 
oscar_1.findAll('span', attrs = {'name' : 'nv'})[1].text

u'$92.05M'

`Not all movies have gross earned so lets add a helper function to get count and gross with default`

In [27]:
def votes_and_gross_conveter(oscar):
    oscar = oscar.find_all('span',{"name":"nv"})
    votes_and_gross_list = []
    for data_value in oscar:
        votes_and_gross_list.append(data_value.text)
    if(len(oscar)==2):
        votes=votes_and_gross_list[0]
        gross = votes_and_gross_list[1]
    else:
        votes=votes_and_gross_list[0]
        gross = None
    
    return votes,gross

In [28]:
votes_and_gross_conveter(oscar_1)

(u'492,917', u'$92.05M')

**metascore**

![metascore](https://i.imgur.com/OKGdSKp.png)


In [29]:
movie_mscore = oscar_1.find('span', class_ = 'metascore').text
movie_mscore.strip()

u'81'

In [30]:
def meta_score(oscar):
    # for most of the movies metascore is not available and for those default to 0
    if oscar.find('span', class_ = 'metascore favorable') is not None:
        meta = oscar.find('span', class_ = 'metascore favorable').text
    else:
        meta = '0'
    return int(meta)

In [31]:
meta_score(oscar_1)

81

### Extract and combine data from multiple pages from the portal

**Now that all of the code block is almost there, need to figure out on how to roll through pages to get all the Oscar movies for the sample number of years** \
*Sample of years chosen is 2000-2021 as the dataset is going to be huge and not to irk IMDM* \
Here are the urls: \
start: https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100  
second: https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100&start=101&ref_=adv_nxt  
last: https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100&start=201&ref_=adv_nxt
  

In [32]:
# assign the start page as the start_url and see if the function returns the pages
url = 'https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100'

In [33]:
# lets do a helper function to get all the pages for 3 pages
def all_oscar_page(start_url):
    all_urls = [] #list to get all urls
    url = start_url # begin page
    while(url != None):  #Loop around all the required webpages and terminates when last page arive!
        all_urls.append(url) # add to the list
        soup = BeautifulSoup(requests.get(url).text,"html.parser") # parser
        
        # this step is where we get next link - in the screenshot below look at the html inspection on 'Next button'
        # use the class function what makes the page to next
        next_links = soup.find_all(class_='lister-page-next next-page') #Extracts the next page link.
        if (len(next_links) == 0):         # If their is no next page, it returns 0.
            url = None
        else:
            next_page = "https://www.imdb.com" + next_links[0].get('href')
            url = next_page
            print(url)
    return all_urls

**Inspect HTML for next button**

![roll_thru_pages](https://i.imgur.com/spaXL4P.png)

In [34]:
all_oscar_page(url) # looks like the function is returning correct links!

https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100&start=101
https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100&start=201
https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100&start=301


['https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100',
 u'https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100&start=101',
 u'https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100&start=201',
 u'https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100&start=301']

## Now that we have all the code together, lets glam up a little and tidy it up
###### Compile extracted information into python data objects

**lets get the libraries**

In [35]:
import requests #packages that is used to download the content from web
import urllib # packages that is used to work with URL libraries
import requests #package built to make HTTP requests user friendly
import os # package used for file process
import re # package for regular expression - best to have it dont know if it is required or not
from bs4 import BeautifulSoup #a Python library for pulling data out of HTML and XML files
import pandas as pd # the omnipresent of all python to work with dataframes
requests.__version__ # check the version of the package

'2.19.1'

### Finally, write and save the info into CSV files

**Final helper function to get the movie detail data file by year for user given start and end year**

In [36]:
def get_oscar_movies(start_year, end_year):    
    start_year = str(start_year) # get the begin year
    end_year = str(end_year) # get the end year
    url = get_oscar_url(start_year, end_year) # helper function to build url for start and end year
    oscar_movies = get_oscar_movie_detail(url) # get all the movie detail from helper function
    # give the columns new titles
    oscar_col_list = ['Title','Year','IMDB Rating','Rated','Duration','Genre','Metascore','Votes','Gross USD']
    oscar_df = pd.DataFrame(oscar_movies, columns = oscar_col_list)
    #create a directory to post the files by year
    os.makedirs('movies_data', exist_ok=True)
    # make the data frame group by year as a dictionary
    oscar_movies_year = {j: oscar_df[oscar_df['Year'] == j] for j in oscar_df['Year'].unique()} 
    #loop thru key, value in dictionary to write the files
    for k, v in oscar_movies_year.items():
        v.to_csv('movies_data\oscar_movies_'+'{}.csv'.format(k), index=None)
    return oscar_df

**A clean looking URL getter than putting together into one big link**

In [37]:
def get_oscar_url(start_year, end_year):    
    base_url = 'https://www.imdb.com/search/title/?'
    release_date = 'release_date='
    sep = ','
    tail_url = '&groups=oscar_winner&sort=year,asc&count=100'
    oscar_url = base_url + release_date + start_year + sep + end_year + tail_url
    return oscar_url

**lets do a helper function to get all the pages between start and end year**


In [38]:
def all_oscar_page(start_url):
    all_urls = [] #list to get all urls
    url = start_url # begin page
    while(url != None):  #Loop around all the required webpages and terminates when last page arive!
        all_urls.append(url) # add to the list
        soup = BeautifulSoup(requests.get(url).text,"html.parser") # parser
        
        # this step is where we get next link - in the screenshot below look at the html inspection on 'Next button'
        # use the class function what makes the page to next
        next_links = soup.find_all(class_='lister-page-next next-page') #Extracts the next page link.
        if (len(next_links) == 0):         # If their is no next page, it returns 0.
            url = None
        else:
            next_page = "https://www.imdb.com" + next_links[0].get('href')
            url = next_page
            #print(url)
    return all_urls

**this is where we go thru each page and also the number of movies in that page**

In [39]:
def get_oscar_movie_detail(url):
    oscar_movies = [] # list of movies of data 
    #url = 'https://www.imdb.com/search/title/?release_date=2000-01-01,2021-12-31&groups=oscar_winner&sort=year,asc&count=100'

    for link in all_oscar_page(url):     #Runs the function for all the pages.
        oscar_soup = BeautifulSoup(requests.get(url).text, 'html.parser') #Extracts out the main html code.
        oscar_containers = oscar_soup.find_all("div",{"class" : "lister-item mode-advanced"}) # get all the containers 
        #print(len(oscar_containers))

        #loop through all the movies in the container to get the attributes
        for oscar in oscar_containers: 
            oscar_movies.append(get_movie_info(oscar))
        return oscar_movies

**getting the movie info**

In [40]:
def get_movie_info(oscar):
    name = oscar.h3.a.text
    year = oscar.h3.find('span', class_ = 'lister-item-year text-muted unbold')
    movie_year = pd.to_numeric(year.text.replace('(','').replace(')','').replace('I',''))
    rating = float(oscar.strong.text)
    certificate = get_certificate(oscar) 
    duration = oscar.find('span', class_ = 'runtime').text
    genre = oscar.find('span', class_ = 'genre').text.strip()
    metascore = meta_score(oscar)
    votes, gross = votes_and_gross(oscar)
    movie_info = [name, movie_year, rating, certificate, duration, genre, metascore, votes, gross]
    return movie_info

**helper functions to get certificate, metascore, votes, gross**

In [41]:
def get_certificate(oscar):
    # cases where there is no certificate assigned - lets give default as Not Rated
    if oscar.find('span', class_ = 'certificate') is not None:
        cert = oscar.find('span', class_ = 'certificate').text
    else:
        cert = 'Not Rated'
    return cert

In [42]:
def meta_score(oscar):
    # for most of the movies metascore is not available and for those default to 0
    if oscar.find('span', class_ = 'metascore favorable') is not None:
        meta = oscar.find('span', class_ = 'metascore favorable').text
    else:
        meta = '0'
    return int(meta)

In [45]:
def votes_and_gross(oscar):
    oscar = oscar.find_all('span',{"name":"nv"})
    votes_and_gross_list = []
    for data_value in oscar:
        votes_and_gross_list.append(data_value.text)
    if(len(oscar)==2):
        votes=votes_and_gross_list[0]
        gross = votes_and_gross_list[1]
    else:
        votes=votes_and_gross_list[0]
        gross = None   
    return votes,gross

**Test the final functions**

In [47]:
oscar_df = get_oscar_movies(2019,2020)

TypeError: makedirs() got an unexpected keyword argument 'exist_ok'

In [ ]:
oscar_df[:5]

As a good neighbor, giving the credit where it is due, I have gained my learning from the following links, my appreciation to all of the folks on the world to provide a great content for others to learn.

Useful Links:\
https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/ \
https://www.dataquest.io/blog/web-scraping-beautifulsoup/

**Finally, commit to Jovian with files**

In [ ]:
jovian.submit(assignment="zerotoanalyst-project1")

In [ ]:
jovian.commit(files=['movies_data\oscar_movies_2019.csv'])

In [ ]:
jovian.commit(files=['movies_data\oscar_movies_2020.csv'])